### Шаблоны

##### Шаблоны классов

##### Какая идея стоит за шаблонами
Ранее мы познакомились с возможностью перегрузки функций. Давайте вспомним её на примере swap:

```c++
// поменять местами два int
void my_swap(int& a, int& b)
{
    int tmp = a;
    a = b;
    b = tmp;    
}

// поменять местами два short
void my_swap(short& a, short& b)
{
    short tmp = a;
    a = b;
    b = tmp;
}

// поменять местами два float
void my_swap(float& a, float& b)
{
    float tmp = a;
    a = b;
    b = tmp;
}

...
```

```c++
class Matrix
{
    double* data_;
};
```

```c++
class MatrixDouble
{
    double* data_;
};

class MatrixInt
{
    int* data_;
};
```


Для решения проблем дублирования кода придуманы шаблоны:

### Решение

```c++
// напишем шаблон - как должна выглядеть функция
template<typename T>
void my_swap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}
```

```c++
int a = 3, b = 5;

// вызов my_swap(int&, int&), тип T указывается программистом явно
my_swap<int>(a, b);

// вызов my_swap(int&, int&), тип T выводится компилятором автоматически
my_swap(a, b); 

float x = 3.f, y = 5.f;
my_swap(x, y);
my_swap<float>(x, y);
```





```c++
template <class T>
class Matrix
{
    T* data_;
};
```

```c++
Matrix<double> m;
Matrix<int> m;
```


##### Шаблоны функций

```c++
template <class T>
void printLine(const T& value)
{
    std::cout << value << '\n';
}
```
```c++
printLine<int>(5);
```


Компилятор может самостоятельно вывести тип шаблона в зависимости от аргументов вызова.

```c++
printLine(5);
```

Важное свойство шаблонов по сравнению с перегрузкой функций:
* шаблон компилируется только тогда, когда он вызывается
* шаблон компилируется только для тех типов, для которых он вызывается.

_и в каждом cpp-файле шаблон компилируется снова и снова_

Показать пример на godbolt.org, позакомментировать функции, продемонстрировать разницу в выхлопе компилятора. https://godbolt.org/z/fM11joGzs


```c++
#include <string>

template<typename T>
void __attribute__ ((noinline)) myswap(T& a, T& b)
{
    T tmp = a;
    a = b;
    b = tmp;
}

int main()
{
    int i1 = 3, i2 = 5;
    myswap(i1, i2);

    float f1 = 3.f, f2 = 5.f;
    myswap(f1, f2);

    double d1 = 3., d2 = 5.;
    myswap(d1, d2);

    std::string s1 = "abc", s2 = "def";
    myswap(s1, s2);

    return 0;
}
```

Особенности шаблонов по сравнению с перегруженными функциями:
* компилируется только то, что инстанциируется в коде
* компилируется столько раз, в скольки единицах трансляции инстанциируется:
    * можно в одном cpp-файле 10 раз позвать myswap(int&, int&) - эта функция скомпилируется единожды
    * можно в 10 cpp-файлах один раз позвать myswap(int&, int&) - эта функция скомпилируется 10 раз
* накладные расходы во время компиляции на кодогенерацию при истанциации
* позволяет компилятору агрессивнее оптимизировать. Раскомментировать `__attribute__((noinline))` из примера и показать какой код сгенерирует компилятор. Объяснить, почему.
* позволяет нарушать ODR

```C++
// my-swap.h
template <typename T>
void my_swap(T& a, T& b);

// my_swap.cpp
...

// main.cpp
#include "my_swap.h"
my_swap<int>(x,y); // err

```

__Вопросы__:
* Где поместить шаблонную функцию, которую нужно использовать в разных cpp-файлах?
* Где поместить её реализацию?
* Может ли шаблонная функция содержать некомпилирующийся код?

Коротко:

* (+) меньше кода
* (+) быстрее
* (-) дольше компилируется
* (-) сложнее писать


##### class или typename

```c++
template <class T>
void printLine(const T& value)
{
}
```
```c++
template <typename T>
void printLine(const T& value)
{
}
```

Никакой разницы.


#### Инстанцирование шаблона

Инстанцирование шаблона – это генерация кода функции или класса по шаблону для конкретных параметров.

```c++
template <class T>
bool lessThan7(T value) { return value < (T)7.5; }
```

```c++
lessThan7(5); //  Инстанцирование
// bool lessThan7(int value) { return value < (int)7.5; }

lessThan7(5.0); //  Инстанцирование
// bool lessThan7(double value)  { return value < (double)7.5; }
```


##### Явное указание типа

```c++
lessThan7<double>(5); //  Инстанцирование
// bool print(float value) { return value < (float)7.5; }
```


#### Константы как аргументы шаблона
Во-первых, шаблон может иметь несколько параметров, а во-вторых, параметры не обязаны быть типами. Они могут быть, например, целыми числами:

```c++
template <class T, size_t Size>
class Array
{
    T data_[Size];
};
```

```c++
Array<int, 5> a;
```

```c++
template<int N, typename T>
T add_value(T x)
{
    return x + N;
}

int a = add_value<5, int>(100);
int a = add_value<5>(100);

// 1. шаблон специализирован программистом частично, тип Т компилятор определит сам
// 2. параметром шаблона выступает целое число.
```


#### Ограничения на параметры шаблона не являющиеся типами

Так можно:

```c++
template <int N>
int foo()
{
    return N * 2;
}
```

А double нельзя:

```c++
template <double N> // Ошибка
void foo()
{
}
```

float тоже нельзя.

> Причины исторические, почему не исправлено до сих пор не знаю.


### Параметры шаблона должны быть известны на этапе компиляции.

```c++
template <int N>
void foo() { }

int x = 3;
foo<x>(); // Ошибка
```


Константы на литералы можно:

```c++
template <int N>
void foo() {}

const int x = 3;
foo<x>(); // Ok
```



А с обычной константой нельзя:

```c++
int bar() { return 0; }

template <int N>
void foo() { }

const int x = bar();
foo<x>(); // Ошибка
```



Но если вычислять значение во время компиляции, то можно:

```c++
constexpr int bar() { return 0; }

template <int N>
void foo() {}

const int x = bar();
foo<x>(); // Ok
```


> constexpr говорит компилятору, что надо стараться вычислить значение на этапе компиляции



Нельзя использовать объекты класса:

```c++
struct A {};

template <A a> // Ошибка
void foo()
{
}
```

Можно указатель на const char:

```c++
template <const char* s>
void foo()
{
}
```


И это даже можно инстанцировать nullptr или 0:

```c++
foo<nullptr>();
foo<0>();
```

Но нельзя литералом:

```c++
foo<"some text">(); // Ошибка
```


#### Параметры шаблона по умолчанию

```c++
template <class X, class Y = int>
void foo()
{
}

foo<char>();
```

```c++
template <class T, class ContainerT = std::vector<T>>
class Queue
{
    ContainerT data_;
};

Queue<int> queue;
```



### Специализация шаблона

```c++
template <class T>
class Vector
{
    ...
}

template <>
class Vector<bool>
{
    ...
};
```


Суммирование последовательности от n, до 0:

```c++
#include <iostream>

template <int n>
int sum();

template <>
int sum<0>() { return 0; }

// template <>
// int sum<1>() { return 1; }
// template <>
// int sum<2>() { return 2 + 1; }
// ...

template <int n>
int sum()
{
    return n + sum<n - 1>();
}

int main()
{
    std::cout << sum<3>() << '\n';
    return 0;
}
```

```nasm
int sum<0>():
        mov     eax, 0
        ret
main:
        call    int sum<3>()
        call    operator<<(int)
        ret
int sum<3>():
        call    int sum<2>()
        add     eax, 3
        ret
int sum<2>():
        call    int sum<1>()
        add     eax, 2
        ret
int sum<1>():
        call    int sum<0>()
        add     eax, 1
        ret
```



#### Разбухание кода

Необдуманное использование шаблонов может привести к разбуханию кода, кода становится много, он перестает помещаться в кеш, что ведет к существенным издержкам.


##### Еще реализация суммирования

```c++
template <int n>
struct sum;

template <>
struct sum<0>
{
    static constexpr int value = 0;
};

template <int n>
struct sum
{
    static constexpr int value = n + sum<n - 1>::value;
};

int main()
{
    std::cout << sum<3>::value << '\n';
    return 0;
}
```

```nasm
main:
        mov     esi, 6
        call    operator<<(int)
        ret
```


##### Вычисления времени компиляции

А стоит ли?

1. Сложно для понимания и поддержки
2. Замедляет компиляцию

> Вероятно лучшей альтернативой будет скрипт делающий вычисления и генерирующий С++ код из констант с рассчитаными значениями.


### Шаблоны операторов

```c++
// N-мерный вектор из курса линейной алгебры типа T
template<typename T, int N>
struct VectorN
{
    T data[N];
};

// в качестве примера запишем операции сложения и умножения для таких векторов

template<typename T, int N>
VectorN<T, N> operator +(const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    VectorN<T, N> rv;
    for (int i = 0; i < N; ++i)
        rv.data[i] = l.data[i] + r.data[i];
    return rv;    
}

template<typename T, int N>
T operator * (const VectorN<T, N>& l, const VectorN<T, N>& r)
{
    T rv = 0;
    for (int i = 0; i < N; ++i)
        rv += l.data[i] * r.data[i];
    return rv;
}
```


### Псевдонимы типов

Старый способ:

```c++
typedef int Seconds;
typedef Queue<int> IntegerQueue;

Seconds i = 5;
IntegerQueue j;
```

Новый (рекомендуемый) способ:

```c++
using Seconds = int;
using IntegerQueue = Queue<int>;

Seconds i = 5;
IntegerQueue j;
```


##### Псевдонимы типов для шаблонов:

```c++
template <class T>
using MyQueue = Queue<T, std::deque<T>>;

MyQueue<int> y;
```


#### Новый синтаксис функций

```c++
auto foo() -> void
{
}
```


#### auto

Позволяет статически определить тип по типу выражения.

```c++
auto i = 5;
auto j = foo();
```



#### range-based for и auto

```c++
for (auto i : { 1, 2, 3 })
    std::cout << i;
```

```c++
for (auto& i : data)
    i.foo();
```



#### decltype

Позволяет статически определить тип по типу выражения.

```c++
int foo() { return 0; }

decltype(foo()) x = 5;
// decltype(foo()) -> int
// int x = 5;
```

```c++
void foo(decltype(bar()) i)
{
}
```



#### Определение типа аргументов шаблона функций

```c++
template <typename T>
T min(T x, T y)
{
    return x < y ? x : y;
}

min(1, 2); // ok
min(0.5, 2); // error
min<double>(0.5, 2); // ok
```


```c++
template <typename X, typename Y>
X min(X x, Y y)
{
    return x < y ? x : y;
}

min(1.5, 2); // что будет тут?
min(1, 0.5); // что будет тут?
```
<details>
<summary>Ответ 1 балл (0.5 ; 0.5)</summary>
min(1.5, 2); // ok (потому что возвращаемое значение Х)
min(1, 0.5); // ne ok (потому что возвращаемое значение Х, при возврате Y будет обрезано)
</details>

#### как решить проблему возвращаемого типа?
<details>
<summary>Ответ 2 балла</summary>
```c++
template <typename X, typename Y>
auto min(X x, Y y) -> decltype(x + y)
{
  return x < y ? x : y;
}

min(1.5, 2); // ok
min(1, 0.5); // ok
```
как раз новый синтаксис со стрелочкой нужен именно поэтому, ведь тип возвращаемого значения должен быть известен сразу, если не пользоваться стрелочкой.
</details>


#### typename

```c++
struct String
{
    using Char = wchar_t;
};

template <class T>
class Parser
{
    T::Char buffer[]; // Ошибка
};
```


Если компилятор встречая идентификатор в шаблоне, может его трактовать как тип или что-то иное (например, как статическую переменную), то он выбирает иное.



```c++
struct String
{
    using Char = wchar_t;
};

template <class T>
class Parser
{
    typename T::Char buffer[]; // Ok
};
```


##### Ошибка инстанцирования шаблона

```c++
template <class T>
bool lessThan7(T value) { return value < 7; }
```

```c++
struct A {};
A a;
lessThan7(a); //  Инстанцирование
// bool print(A value)  { return value < 7; }
// Ошибка инстанцирования, тип а не имеет operator<(int)
```

Пример шаблонного класса:

Напишем свой собственный `optional`. Он будет не так хорош, как `std::optional`, потому что мы не знаем пока всех необходимых трюков С++, но для начала пойдёт.

Идея `optional` - класс-обёртка над значением, которое может отсутствовать.

Для начала объявим шаблонный класс:

```c++
template<typename T>
class Optional {
    ...
};
```

Добавим в класс поля:

```c++
template<typename T>
class Optional {
    
private:
    T value_;
    bool has_value_;
    
    ...
};
```

**Вопрос:** Мы выбрали способ хранения объекта как поле класса, потому что так проще для демонстрации. Какие у него недостатки? Как их обойти?

<details>
<summary>Ответ 0.5</summary>

* Недостаток - объект существует даже когда он не нужен
* Вариант обхода 1 - хранить объект в куче через new. Проблема: расходы на new/delete.
* Вариант обхода 2 - кусок сырой памяти под объект + placement new и ручной вызов деструкторов (трюк в std::optional)

</details>

Добавим конструкторы:
    
```c++
template<typename T>
class Optional {
    ...
        
public:
    Optional()
        : has_value_(false)
    {}
        
    Optional(const T& another_value)
        : value_(another_value)
        , has_value_(true)
    {}   
};

//
// usage:
//
Optional<std::string> maybe_string_1;
Optional<std::string> maybe_string_2("hello world");
```

**Вопрос:** Почему `const& `   у   ` another_value` ?

**Вопрос (0.35 балла):** Что мы должны срочно добавить в класс?

Срочно добавим:


```c++
template<typename T>
class Optional {
    ...

public:
    Optional(const Optional&) = default;
    Optional(Optional&&) = default;
    Optional& operator = (const Optional&) = default;
    Optional& operator = (Optional&&) = default;
    ~Optional() = default;
};

//
// usage:
//
Optional<std::string> maybe_string_1("hello world");
Optional<std::string> maybe_string_2 = maybe_string_1;
Optional<std::string> maybe_string_3 = std::move(maybe_string_1);
```

Добавим немного функционала:

```c++
template<typename T>
class Optional {
    ...
        
public:
    bool has_value() const { return has_value_; }
    
    T&       get_value()       { return value_; }
    const T& get_value() const { return value_; }
    
    T*       get_ptr()       { return has_value_ ? &value_ : nullptr; }    
    const T* get_ptr() const { return has_value_ ? &value_ : nullptr; }
    
    void reset() {
        value_ = T();
        has_value_ = false;
    }

    void reset(const T& another_value) {
        value_ = another_value;
        has_value_ = true;
    }
    
    void emplace() {
        value_ = T();
        has_value_ = true;
    }
};

//
// usage:
//
Optional<std::string> maybe_string_1("hello world");

if (maybe_string_1.has_value())
    std::cout << maybe_string_1.get_value() << std::endl;

if (std::string* s = maybe_string_1.get_ptr())
    std::cout << *s << std::endl;

maybe_string_1.reset();

maybe_string_1.emplace();
```

Добавим операторы сравнения:
    
```c++
template<typename T>
class Optional { ... };

template<typename T>
bool operator == (const Optional<T>& lhs, const Optional<T>& rhs) {
    if (!lhs.has_value())
        return !rhs.has_value();
    
    if (!rhs.has_value())
        return false;
    
    return lhs.get_value() == rhs.get_value();    
}

template<typename T>
bool operator != (const Optional<T>& lhs, const Optional<T>& rhs) {
    return !(lhs == rhs);
}

template<typename T>
bool operator == (const Optional<T>& lhs, const T& rhs) {
    return
        lhs.has_value() &&
        lhs.get_value() == rhs;
}

template<typename T>
bool operator == (const T& lhs, const Optional<T>& rhs) {
    return rhs == lhs;
}

template<typename T>
bool operator != (const Optional<T>& lhs, const T& rhs) {
    return !(lhs == rhs);
}

template<typename T>
bool operator != (const T& lhs, const Optional<T>& rhs) {
    return !(lhs == rhs);
}



//
// usage:
//
Optional<std::string> maybe_string_1("hello world");
Optional<std::string> maybe_string_2 = maybe_string_1;

if (maybe_string_1 != maybe_string_2)
    std::cout << "unreachable!" << std::endl;

Optional<int> maybe_int;

if (maybe_string_1 == maybe_int)  // compile-time error!
    std::cout << "unreachable!" << std::endl;
```

**Вопрос:** какая проблема в этом коде?

```c++
Optional<std::string> maybe_string("hello world");

if (maybe_string == "C++ is designed for faster code")
    std::cout << "Fast enough?" << std::endl;
```

Решим эту проблему, добавим ещё один более хитрый оператор сравнения:
    
```c++

template<typename T, typename U>
bool operator == (const Optional<T>& lhs, const U& rhs) {
    return
        lhs.has_value() &&
        lhs.get_value() == rhs;
}

template<typename T, typename U>
bool operator == (const T& lhs, const Optional<U>& rhs) {
    return rhs == lhs;
}

template<typename T, typename U>
bool operator != (const Optional<T>& lhs, const U& rhs) {
    return !(lhs == rhs);
}

template<typename T, typename U>
bool operator != (const T& lhs, const Optional<U>& rhs) {
    return !(rhs == lhs);
}

//
// usage:
//

Optional<std::string> maybe_string_1("hello world");

if (maybe_string == "C++ is designed for faster code")
    std::cout << "Fast enough?" << std::endl;
```

Чтобы не было скучно, добавим в шаблонный класс шаблонный метод.


```c++
template<typename T>
class Optional {
    ...

public:
    // уже было
    void emplace(const T& another_value) {
        value_ = another_value;
        has_value_ = true;
    }
    
    // добавили
    template<typename U>
    void emplace(const U& source) {
        value_ = source;
        has_value_ = true;
    }
};

//
// usage:
//
Optional<std::string> maybe_string;
maybe_string.emplace("hello world");
```

**Вопрос:** Что поменялось при вызове `emplace`?

Самое время добавить в класс шаблонный конструктор!

```c++
template<typename T>
class Optional {
    ...
        
public:
    // уже было
    Optional(const T& another_value)
        : value_(another_value)
        , has_value_(true)
    {}

    // добавили
    template<typename U>
    Optional(const U& source)
        : value_(source)
        , has_value_(true)
    {}
};

//
// usage:
//
Optional<std::string> maybe_string("hello world");
```

**Вопрос:** Что поменялось в вызове конструктора от `const char*` ?

**Вопрос:** Можем ли мы что-нибудь выиграть от шаблонного деструктора ?

Пример целиком без операторов.

```c++
#include <string>

template<typename T>
class Optional {
private:
    T value_;
    bool has_value_;

public:
    Optional()
        : has_value_(false)
    {}

    Optional(const T& another_value)
        : value_(another_value)
        , has_value_(true)
    {}

    template<typename U>
    Optional(const U& source)
        : value_(source)
        , has_value_(true)
    {}

    Optional(const Optional&) = default;
    Optional(Optional&&) = default;
    Optional& operator = (const Optional&) = default;
    Optional& operator = (Optional&&) = default;
    ~Optional() = default;

    bool has_value() const { return has_value_; }

    T&       get_value()       { return value_; }
    const T& get_value() const { return value_; }

    T*       get_ptr()       { return has_value_ ? &value_ : nullptr; }    
    const T* get_ptr() const { return has_value_ ? &value_ : nullptr; }

    void reset() {
        value_ = T();
        has_value_ = false;
    }

    void reset(const T& another_value) {
        value_ = another_value;
        has_value_ = true;
    }

    void emplace() {
        value_ = T();
        has_value_ = true;
    }

    template<typename U>
    void emplace(const U& source) {
        value_ = T(source);
        has_value_ = true;
    }
};


std::string f() {
    Optional<std::string> opt;

    opt.emplace("hello");
    opt.emplace("C++");
    opt.emplace("world");        

    opt.emplace(std::string("-hello"));
    opt.emplace(std::string("-C++"));
    opt.emplace(std::string("-world"));

    return opt.get_value();
}
```

# Доп материал по шаблонам


#### SFINAE (Substitution Failure Is Not An Error)

При определении перегрузок функции ошибочные инстанциации шаблонов не вызывают ошибку компиляции, а отбрасываются из списка кандидатов на наиболее подходящую перегрузку.

Неудачное инстанцирование шаблона - это не ошибка.

Например, позволяет на этапе компиляции выбрать нужную функцию:




```c++
// C++11

template<typename T>
void clear(T& t, 
    typename std::enable_if<std::is_pod<T>::value>::type* = nullptr)
{
    std::memset(&t, 0, sizeof(t));
}

// Для не-POD типов
template<typename T>
void clear(T& t, 
    typename std::enable_if<!std::is_pod<T>::value>::type* = nullptr)
{
    t = T{};
}
```



##### is_pod

```c++
template <class T>
struct is_pod
{
    static constexpr bool value = false;
};
```

```c++
template <>
struct is_pod<int>
{
    static constexpr bool value = true;
};
```



##### enable_if

```c++
template<bool, typename T = void>
struct enable_if 
{
};

// Частичная специализация для true
template<typename T>
struct enable_if<true, T>
{
    using type = T;
};

enable_if<false, int>::type // Ошибка, нет type
enable_if<true, int>::type // Ок, type == int
```


```c++
// C++14

template<typename T>
void clear(T& t, std::enable_if_t<std::is_pod<T>::value>* = nullptr)
{
    std::memset(&t, 0, sizeof(t));
}

// Для не-POD типов
template<typename T>
void clear(T& t, std::enable_if_t<!std::is_pod<T>::value>* = nullptr)
{
    t = T{};
}
```



Можно получить на этапе компиляции информацию о типе, например, проверим есть ли у класса некий метод:

```c++
struct A
{
    void foo() {}
};

struct B
{
};

template<typename T>
struct HasFoo
{
    static constexpr bool value = true;
};

int main()
{
    std::cout << hasFoo<A>::value << '\n';
    std::cout << hasFoo<B>::value << '\n';
    return 0;
}
```

```c++
template<typename T>
struct HasFoo
{
    static constexpr bool value = ???;
};
```


Нам нужно будет 2 функции: одна принимает класс с нужным нам методом, другая принимает все остальное:

```c++
template<typename T>
struct HasFoo
{
    // Принимает все
    static int check(...);

    // Принимает нужный нам класс, 
    // где есть какая-то foo()
    template <class U>
    static auto check(U* u) -> decltype(u->foo());
};
```

По возвращаемому функцией типу мы поймем, какая из перегрузок была использована, если тип совпадет, то это то, что нам нужно.

Проверка совпадения типов:

```c++
template <class T1, class T2>
struct IsSame
{
    static constexpr bool value = false;
};

template <class T>
struct IsSame<T, T>
{
    static constexpr bool value = true;
};
```

Финальный вариант:

```c++
template<typename T>
struct HasFoo
{
private:
    static int check(...);

    template <class U>
    static auto check(U* u) -> decltype(u->foo());

public:
    static constexpr bool value = 
        IsSame
        <
            void, 
            decltype(HasFoo<T>::check((T*) nullptr))
        >::value;
};
```

```c++
hasFoo<A>::value == true;
hasFoo<B>::value == false;
```


#### type_traits

В стандартной библиотеки функции определения свойств типов is_* находятся в заголовочном файле type_traits

Примеры:

```c++
is_integral  // Является ли тип целочисленным 
is_floating_point // Является ли тип типом с плавающей точкой
is_array // Является ли тип типом массива
is_const  // Содержит ли тип в себе квалификатор const
is_pod // Является ли тип POD-типом
has_virtual_destructor // Имеет ли виртуальный деструктор

// И так далее
```

### Шаблоны свойств (traits)

```c++
template <typename T>
struct NumericTraits
{
};

template <> // Специализация
struct NumericTraits<char>
{
    static constexpr int64_t min = -128;
    static constexpr int64_t max = 127;
};
```



```c++
template <typename T>
class Calculator
{
    T getNumber(const std::string& text)
    {
        const int64_t value = toNumber(text);
        if (value < NumericTraits<T>::min
            || value > NumericTraits<T>::max)
        {
            // range error
        }
    }
};
```

> Смотрите заголовочный файл numeric_limits

Не только значения, но и типы:

```c++
template <class T>
class BasicStream
{
public:
    using Char = T;
};

using Utf8Stream = BasicStream<char>;

Utf8Stream::Char c;
```


### Классы стратегий

Класс стратегий - интерфейс для применения стратегий в алгоритме.

```c++
class Json
{
public:
    void encode(const char* data, size_t size) {}
};

class Xml
{
public:
    void encode(const char* data, size_t size) {}
};

template <class T, class Format>
class Connector
{
    Format format_;
public:
    void connect()
    {
        auto packet = makeConnectPacket();
        auto encodedPacket = format_.encode(
            packet.data, packet.size);
        send(encodedPacket);
    }
};

template<class T>
using JsonConnector = Connector<T, Json>;
```


### Отличия между свойствами и стратегиями

Свойство - отличительная особенность характеризующая сущность.

Стратегия - образ действия сущности.


### Сравнение динамического и статического полиморфизма

```c++
class Device
{
public:
    virtual ~Device() {}

    virtual void write(const char* data, size_t size) = 0;
};

class File final
    : public Device
{
public:
    void write(const char* data, size_t size) override {}
};

class Stream
{
    Device* device_;
public:
    explicit Stream(Device* device)
        : device_(device)
    {
    }
    
    void putChar(char c)
    {
        device_->write(&c, 1);
    }
};

auto stream = Stream(new File("file.txt"));
```
```c++
class File
{
public:
    explicit File(const char* name) {}
    void write(const char* data, size_t size) {}
};

template <class Device>
class Stream
{
    Device device_;
public:
    explicit Stream(Device&& device)
        : device_(std::move(device))
    {
    }

    void putChar(char c)
    {
        device_.write(&c, 1);
    }
};

using FileStream = BasicStream<File>;

FileStream stream(File("data"));
```

1. Динамический полиморфизм более гибок и позволяет настраивать поведение во время выполнения, но имеет накладные расходы на вызов виртуальных методов
2. Статический полиморфизм не имеет накладных расходов, но менее гибок
